In [11]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from collections import OrderedDict
from dataset.mnist import load_mnist
from common.gradient import numerical_gradient
from common.functions import softmax, cross_entropy_error

# 곱셈 계층
class MulLayer:
    def __init(self):
        self.x = None
        self.y = None
        
    # 순전파 
    def forward(self, x, y):
        self.x = x
        self.y = y
        
        return x * y
    
    # 역전파, 상류에서 넘어온 미분에 순전파 때의 입력과 출력값을 바뀌서 곱한뒤 하류로 흘린다.
    def backward(self, dout): # dout = 상류에서 넘어온 미분값, 맨 처음엔 순전파 때의 결과값의 미분값
        dx = dout * self.y
        dy = dout * self.x
        
        return  dx, dy
    
# 덧셈 계층
class AddLayer:
    def __init__(self):
        pass # 아무런 실행 없이 넘김
    
    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy
    
# ReLU 계층(활성화 함수)
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = x <= 0 # 인수 X(넘파이 배열)의 원소를 하나씩 조건에 맞게 True나 False로 변환
        out = x.copy()
        out[self.mask] = 0 # 인덱스 값으로 True가 들어가면 해당값 추가, False가 들어가면 해당 값은 뺀다.
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx
    
# Sigmoid 계층(활성화 함수)
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out # 순전파의 출력을 인스턴스 변수 out(self.out)에 저장했다가 역전파 계산 때 그 값을 사용한다.
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx
    
# Affine 계층(합성곱 계층), 기하학에서 신경망의 순전파 때 수행하는 행렬의 곱을 의미한다.
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T) # numpy 배열에 .T 속성을 주면 전치연산처리가 되어서 원래 배열의 행과 열이 바뀐다
        self.dW = np.dot(self.x.T, dout) # 행렬의 곱 계산
        self.db = np.sum(dout, axis=0) # 각 행렬의 원소를 대치되는 인덱스를 가진 원소끼리 더한다.
        
        return dx

# Softmax-with-Loss 계층, 입력 값을 정규화하여 출력한다.
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None # softmax의 출력
        self.x = None # 정답 레이블(one-hot 벡터)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx
    
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성, 2층
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu() # 활성화 계층(함수)
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x) # 각 계층의 순전파 실행
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y ,t) # softmax의 손실함수 구하기
    
    # 정확도(예측값(출력값)과 정답 사이의 수치) 계산
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x ,t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout) # 각 계층의 역전파 실행
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        return grads
    
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10) # 신경망 생성

iters_num = 10000 # 전체 입력 수
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1) # 에폭의 단위

for i in range(iters_num):
    # 훈련 모델 중 배치 수 만큼 무작위 추출
    batch_mask = np.random.choice(train_size, batch_size)
    # batch_mask 에서 무작위 추출된 값들이 인덱스가 되고 그 해당 인덱스에 맞는 값을 제외하곤 나머진 0이 됨
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key] # 오차역전파법으로 기울기 구하면 각 매개변수에 구한 기울기 X 학습률을 빼서 갱신
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1 에폭 만큼 반복했을 시
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

"""
x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    # 수치 미분과 오차역전파법의 결과 오차가 0이 되는 일은 드물다, 즉 값이 작아질 수록 좋다.
    diff = np.average(np.abs(grad_backprop[key] - grad.numerical[key]))
    print(key + ":" + diff)
"""

0.13125 0.1317
0.9034833333333333 0.9069
0.9229 0.9237
0.9338333333333333 0.9339
0.9444666666666667 0.9437
0.9495333333333333 0.9475
0.9568666666666666 0.9563
0.9613666666666667 0.958
0.96315 0.9587
0.9685333333333334 0.9652
0.97055 0.9657
0.97125 0.9666
0.9729833333333333 0.9677
0.9733666666666667 0.9659
0.9762333333333333 0.9683
0.9769833333333333 0.97
0.9776833333333333 0.9685


'\nx_batch = x_train[:3]\nt_batch = t_train[:3]\n\ngrad_numerical = network.numerical_gradient(x_batch, t_batch)\ngrad_backprop = network.gradient(x_batch, t_batch)\n\n# 각 가중치 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.\nfor key in grad_numerical.keys():\n    # 수치 미분과 오차역전파법의 결과 오차가 0이 되는 일은 드물다, 즉 값이 작아질 수록 좋다.\n    diff = np.average(np.abs(grad_backprop[key] - grad.numerical[key]))\n    print(key + ":" + diff)\n'